In [1]:
urls = [' https://raw.githubusercontent.com/CriMenghini/ADM/master/2019/Homework_3/data/movies3.html',
        'https://raw.githubusercontent.com/CriMenghini/ADM/master/2019/Homework_3/data/movies2.html',
        'https://raw.githubusercontent.com/CriMenghini/ADM/master/2019/Homework_3/data/movies1.html']

# extract urls and save :Article_i.html

In [ ]:
import requests
from bs4 import BeautifulSoup
import random
import time
movie_links = []
for url in urls:
  response = requests.get(url)
  soup = BeautifulSoup(response.text, 'html.parser')
  for link in soup.find_all('a', href = True):
    movie_links.append(link['href'])
for movie in range (30000+1):
  response_page = requests.get(movie_links[movie])
  if int(response_page.status_code)==200:
    time.sleep(random.randint(1,5))
  elif  int(response_page.status_code)==404: # Not Found
    print ('url not found')
    time.sleep(random.randint(1,5))
  elif int(response_page.status_code)!=429: #Too Many Requests
    time.sleep(1500)
  soup = BeautifulSoup(response_page.text, 'html.parser')
  name = "Article_{0}.html".format(movie)
  with open (name, 'w') as file:
    file.write(name)

# extract data include name intro plot and etc

In [ ]:
# here i try with one link for test code 
import requests
from bs4 import BeautifulSoup
from lxml import etree
import lxml.html
import csv
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize

response = requests.get('https://en.wikipedia.org/wiki/10_to_Midnight') 

soup = BeautifulSoup(response.text, 'html.parser')

# find intro location and extract text
intro = soup.select_one('div#bodyContent div#mw-content-text p').text

# find plot location and extract text
plot = soup.select_one('div#bodyContent div#mw-content-text').find_all('p')[1].text

# remove stop words intro
stop_words = set(stopwords.words('english')) 
text_tokens_intro = word_tokenize(intro)
tokens_without_sw_intro = [word for word in text_tokens_intro if not word in stopwords.words()]

# remove stop words plot
text_tokens_plot = word_tokenize(plot)
tokens_without_sw_plot = [word for word in text_tokens_plot if not word in stopwords.words()]

# remove punctuation intro 
tokenizer = nltk.RegexpTokenizer(r"\w+")

punc_intro = ",".join(tokens_without_sw_intro)
new_words_intro = tokenizer.tokenize(punc_intro)

# remove punctuation plot 
punc_plot = ",".join(tokens_without_sw_plot)
new_words_plot = tokenizer.tokenize(punc_plot)


#Stemming intro and plot 

ps = PorterStemmer()
streaming_intro = []
streaming_plot = []
for x in new_words_intro:
    streaming_intro.append(ps.stem(x))
for y in new_words_plot:
    streaming_plot.append(ps.stem(y))


table = soup.find('table', class_ = 'infobox vevent')
element_tree = lxml.html.fromstring(response.text)
check_list = ['movie name','Directed by','Produced by','Written by','Starring','Music by','Release date','Running time','Country','Language','Budget']
info = []
stars ='//*[@id="mw-content-text"]/div[1]/table/tbody/tr[6]/td/div/ul/li/*/text()'
writen ='//*[@id="mw-content-text"]/div[1]/table/tbody/tr[5]/td//a/text()'
Path =[
    '//*[@id="mw-content-text"]/div[1]/table/tbody/tr[1]/th/text()',
    '//*[@id="mw-content-text"]/div[1]/table/tbody/tr[3]/td/a/text()',
    '//*[@id="mw-content-text"]/div[1]/table/tbody/tr[4]/td/text()',
    writen,
    stars,
    '//*[@id="mw-content-text"]/div[1]/table/tbody/tr[7]/td/a/text()',
    '//*[@id="mw-content-text"]/div[1]/table/tbody/tr[12]/td/div/ul/li/text()',
    '//*[@id="mw-content-text"]/div[1]/table/tbody/tr[13]/td/text()',
    '//*[@id="mw-content-text"]/div[1]/table/tbody/tr[14]/td/text()',
    '//*[@id="mw-content-text"]/div[1]/table/tbody/tr[15]/td/text()',
    '//*[@id="mw-content-text"]/div[1]/table/tbody/tr[16]/td/text()'
  
]
for i in Path:
    if element_tree.xpath(i):
        info.append(element_tree.xpath(i))
    else:
        info.append("NA")
  


print("*"*100)
data = []
# data inclue intro plot and check_list
data.append(streaming_intro)
data.append(streaming_plot)
for i in info:
    if len(i)==1:
        data.append(*(i))
    elif len(i) > 1:
        data.append(i)
      
#     elif len(i) > 1:
#         for x in i:
#             new_info.append(x)      
  
with open('output.tsv', 'w', newline='') as f_output:
    tsv_output = csv.writer(f_output, delimiter='\t')
    tsv_output.writerow(data)
     
print(data)